In [ ]:
apptainer shell --nv -B /lscratch/$SLURM_JOBID/Flanker:/Flanker -B /home/nielsond:/home/nielsond -B /lscratch/$SLURM_JOBID/cache:/data/nielsond/.cache ./flanker.sif 

sinteractive --gres=gpu:k80:1,lscratch:20 --mem=40g -c20 --time=24:00:00  

In [2]:
from pathlib import Path

In [3]:
sif_path = Path('/data/MLDSST/nielsond/cogmood/analysis/SUPREME/Analysis/Flanker/flanker.sif')
swarm_dir = Path('/data/MLDSST/nielsond/cogmood/swarm/')
swarm_cmd_dir = swarm_dir / 'cmds'
swarm_log_dir = swarm_dir / 'log'
clean_csv_dir = Path('/data/MLDSST/nielsond/cogmood/data/20250611_pilot/to_model')
res_dir = Path('/data/MLDSST/nielsond/cogmood/data/20250611_pilot/model_res')
res_dir.mkdir(exist_ok=True)

In [49]:
cmds = []
for sub_file in sorted(clean_csv_dir.glob('*.csv'))[3:]:
    cmd = f"""rsync -avch /data/MLDSST/nielsond/cogmood/analysis/SUPREME/Analysis/Flanker /lscratch/$SLURM_JOBID/; \
    rsync -avch {sub_file} /lscratch/$SLURM_JOBID/Flanker/; \
    cd /lscratch/$SLURM_JOBID/Flanker; \
    mkdir -p /lscratch/$SLURM_JOBID/cache/pycuda; \
    ls -ltah /lscratch/$SLURM_JOBID/Flanker/flanker.sif; \
    export MPLBACKEND=pdf; \
    apptainer run --nv -B /lscratch/$SLURM_JOBID/Flanker:/Flanker \
        -B /lscratch/$SLURM_JOBID/cache:/data/nielsond/.cache \
        /lscratch/$SLURM_JOBID/Flanker/flanker.sif {sub_file.parts[-1]} ; \
    rsync -avch flanker_{sub_file.stem}.tgz {res_dir}/; 
    """
    cmds.append(cmd)

In [50]:
cmd_file = swarm_cmd_dir/'pilot_flkr3'
cmd_file.write_text('\n'.join(cmds))

14382

In [51]:

! swarm --time=12:00:00 -g 20 -t 20 -m apptainer --gres=gpu:k80:1,lscratch:5 --partition=gpu --job-name=flkr_pilot --logdir={swarm_log_dir} {cmd_file}

66215664
